# 1. Imports

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification

from codes.classification_codes import krc_gridsearch



# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([T_dict[item] for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))

X = features
Y = np.array(T_target)
groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)

train_val_Y = train_val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')
<class 'generator'>
(235, 100)
(235,)
(27, 100)
(27,)


# 3. Runing Cross-Validation

In [3]:
def mul_range(start, stop, multiple):
    value = start
    while value < stop: # using < for consistency with range
        yield value
        value *= multiple

Alpha = [item for item in mul_range(0.0001, 30, 1.2)]
Max_iter = [90, 100, 120, None]


def cross_10_folds_krc(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        
        model_path = f'classification/ImageNet/aug3/KRC_{fold}_model.sav'
        
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
    
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best = krc_gridsearch(train_X, train_Y, val_X, val_Y, Alpha, Max_iter, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_krc(train_val_X, train_val_Y)
print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')

<class 'generator'>
best_train_acc: 0.9428571428571428, best_val_acc: 0.7083333333333334
fold: 0 done!
<class 'generator'>
best_train_acc: 0.9371069182389937, best_val_acc: 0.7083333333333334
best_train_acc: 0.9339622641509434, best_val_acc: 0.75
best_train_acc: 0.9339622641509434, best_val_acc: 0.7916666666666666
fold: 1 done!
<class 'generator'>
best_train_acc: 0.9465408805031447, best_val_acc: 0.7083333333333334
fold: 2 done!
<class 'generator'>
best_train_acc: 0.946031746031746, best_val_acc: 0.5833333333333334
best_train_acc: 0.9301587301587302, best_val_acc: 0.625
best_train_acc: 0.9333333333333333, best_val_acc: 0.7083333333333334
best_train_acc: 0.926984126984127, best_val_acc: 0.75
best_train_acc: 0.9301587301587302, best_val_acc: 0.7916666666666666
best_train_acc: 0.9238095238095239, best_val_acc: 0.8333333333333334
fold: 3 done!
<class 'generator'>
best_train_acc: 0.9492063492063492, best_val_acc: 0.5833333333333334
best_train_acc: 0.9492063492063492, best_val_acc: 0.625
bes

# 4. Model Testing

In [4]:
best_fold = 3


model_path = f'classification/ImageNet/aug3/KRC_{best_fold}_model.sav'

# Fitting different train in the same hyperparameters
best_test, confusion_matrix_test = model_test_classification(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)



print(best_test)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}')

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
[0.5555555555555556, 0.5925925925925926, 0.5925925925925926, 0.5555555555555556, 0.5185185185185185, 0.48148148148148145, 0.6296296296296297, 0.5555555555555556, 0.5925925925925926, 0.5555555555555556]
test_acc_mean: 0.563, std: 0.040
5th confusion matrix:  [[0, 3, 0], [3, 7, 3], [0, 4, 7]]


In [5]:
cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print([list(item) for item in np.mean(cm_list, axis=0)])

10
[[0.5, 2.5, 0.0], [3.1, 7.4, 2.5], [0.3, 3.4, 7.3]]


In [6]:
best_fold = 3
PATH = f'Models/classification/ImageNet/aug3/KRC_{best_fold}_model.sav'

#PATH = os.path.join('Models', model_path)
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()

{'alpha': 29.07407974435072,
 'class_weight': None,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 90,
 'positive': False,
 'random_state': 0,
 'solver': 'auto',
 'tol': 0.0001}